In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import json
import requests

from CP_trend import expand_months_manual

from pandas.io.json import json_normalize

Following API call will return  weekly  product sku counts & price time series data with floral in the product name and description under women category for Target (r3) for last 3 month
https://market-api-3.trendalytics.co/v1/products/histogram?filters=sku&cat=women&fts=floral&fl=r3&regions=us&pricetype=current&sku_by=week&month_span=3

In [2]:
api_endpt = 'https://market-api-3.trendalytics.co/v1/products/histogram?filters=sku&cat=womens-clothes'\
            '&fts=high%20waisted%20jeans&fl=r58533412-zara&'\
            'regions=us&pricetype=current&sku_by=week&month_span=24'

#            'https://market-api-3.trendalytics.co/v1/products/histogram?filters=sku&cat=women'\
#            '&fts=floral&fl=r3&'\
#            'regions=us&pricetype=current&sku_by=week&month_span=24


response = requests.get(api_endpt)
output = json.loads(response.content.decode('utf-8'))

In [3]:
len(output['categoryHistogram'])

27

In [4]:
[ x['id'] for x in output['categoryHistogram']]

['clothes-shoes-and-jewelry',
 'women',
 'womens-clothes',
 'jeans',
 'skinny-jeans',
 'flare-jeans',
 'straight-leg-jeans',
 'relaxed-jeans',
 'cropped-jeans',
 'kids-and-baby',
 'kids-and-baby-clothing',
 'girls-jeans',
 'distressed-jeans',
 'men',
 'mens-clothes',
 'mens-jeans',
 'stretch-jeans',
 'bootcut-jeans',
 'classic-jeans',
 'mens-straight-leg-jeans',
 'shorts',
 'kids-jeans',
 'boys-jeans',
 'mens-distressed-jeans',
 'mens-relaxed-jeans',
 'wide-leg-pants',
 'womens-pants']

In [5]:
len(output['skuHistogram'])

105

In [7]:
output['skuHistogram'] 

[{'id': '0',
  'name': '2017-10-22 to 10-28',
  'from': '2017-10-22T00:00:00.000Z',
  'to': '2017-10-28T23:59:59.999Z',
  'count': 0,
  'price': [{}]},
 {'id': '1',
  'name': '2017-10-29 to 11-04',
  'from': '2017-10-29T00:00:00.000Z',
  'to': '2017-11-04T23:59:59.999Z',
  'count': 16,
  'price': [{'usd': {'max': 69.9,
     'min': 29.9,
     'sum': 672.3999999999999,
     'avg': 42.02,
     'lower_quartile': 35.9,
     'median': 39.9,
     'upper_quartile': 49.9,
     'count': 16}}]},
 {'id': '2',
  'name': '2017-11-05 to 11-11',
  'from': '2017-11-05T00:00:00.000Z',
  'to': '2017-11-11T23:59:59.999Z',
  'count': 90,
  'price': [{'usd': {'max': 89.9,
     'min': 29.9,
     'sum': 3935,
     'avg': 43.72,
     'lower_quartile': 29.9,
     'median': 39.9,
     'upper_quartile': 49.9,
     'count': 90}}]},
 {'id': '3',
  'name': '2017-11-12 to 11-18',
  'from': '2017-11-12T00:00:00.000Z',
  'to': '2017-11-18T23:59:59.999Z',
  'count': 102,
  'price': [{'usd': {'max': 89.9,
     'min': 29.

In [9]:
output['skuHistogram'][0]['price'][0] == {}

True

In [10]:
def expand_agg_price(skuhist_list, trend_top, trend, retailer):
    '''
    @description: this function expands the dataframe from the API call to include the columns for the   months and the
                prices of each of the products during that month
    @params:
    - skuhis_list: the output of the API call and key 'products'
    - trend: trend to search, i.e. "high waisted jeans"
    - retailer: the retailer name
    - metric: count, median, min, max
    @return: returns the expanded dataframe, single row
    '''
    count_df= pd.DataFrame()
    count_df.at[0,"trend_top"] = trend_top
    count_df.at[0,"trend"] = trend
    count_df.at[0,"ret_id"] = retailer #only 1
    med_df= pd.DataFrame()
    med_df.at[0,"trend_top"] = trend_top
    med_df.at[0,"trend"] = trend
    med_df.at[0,"ret_id"] = retailer #only 1
    month_cols = ['2018-01','2018-02','2018-03','2018-04','2018-05','2018-06','2018-07','2018-08','2018-09',\
                 '2018-10','2018-11','2018-12','2019-01','2019-02','2019-03','2019-04','2019-05','2019-06',\
                  '2019-07','2019-08','2019-09']
    for c in month_cols:
        count_df[c] = np.nan
        med_df[c] = np.nan
        
    for b in skuhist_list:
        if not b['price'][0] == {}:
            
            date = b['from'][:7]
            count = b['count']
            med = b['price'][0]['usd']['median']
            count_df.at[0,date] = count
            med_df.at[0,date] = med
    
    return count_df, med_df
        

In [11]:
count_df_test, med_df_test = expand_agg_price(output['skuHistogram'], 'high waisted jeans', 'Zara')

In [12]:
count_df_test

,trend,ret_id,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2017-10,2017-11,2017-12,2019-10
0,high waisted jeans,Zara,23.0,53.0,65.0,86.0,96.0,76.0,45.0,60.0,...,36.0,40.0,75.0,120.0,107.0,147.0,16.0,91.0,92.0,167.0


In [13]:
med_df_test

,trend,ret_id,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2017-10,2017-11,2017-12,2019-10
0,high waisted jeans,Zara,49.9,49.9,49.9,49.9,49.9,35.99,49.9,49.9,...,44.9,44.9,22.99,29.99,49.9,49.9,39.9,39.9,29.99,49.9


In [14]:
#read retailer ID
ret_df = pd.read_csv('Retailer_ID.csv', encoding = "ISO-8859-1")

In [15]:
ret_df.head(4)

,Market Index,Retail Site,Retailer ID,Crawled?
0,Contemporary Index,Alice + Olivia,864,Crawler
1,Contemporary Index,Isabel Marant,400331576-isabelmarant,Crawler
2,Contemporary Index,A.L.C.,45771179-alc,Crawler
3,Contemporary Index,Sandro,2412,Crawler


In [16]:
ret_df = ret_df.drop(range(104,len(ret_df))) #truncate Off Price Index

In [17]:
ret_df.shape

(104, 4)

In [18]:
ret_df['Retailer ID'].values

array(['864', '400331576-isabelmarant', '45771179-alc', '2412', '801',
       '760', '152651556-acnestudios', '804731108-iro', '1909',
       '750312749-thekooples', '760', '359318405-lagence', '105', '21',
       '1691', '500269677-aritzia', '8', '25', '306', '1', '23',
       '756639511-seedheritage', '574', '930080541-massimodutti',
       '398219112-torrid', '2001', '2294', nan, '1690', '58533412-zara',
       '108', '36', '1745', '483', '408', '1689', '970193273-rue21', '29',
       '1841', '491303298-next', '2351', '2531', nan, '27', '895', '1375',
       '1402', '6', '7', '994', '769926722-hermes', '10', '9', '2601',
       '344', '625324324-louisvuitton', '924989490-prada',
       '562481900-saintlaurent', '312443187-valentino', '2493',
       '157913149-tomford', '900125254-chloe', '777583743-dolcegabbana',
       '220138893-dior', '1787', '494', '1340', '716752426-walmart',
       '928', '413', '2', '133', '150', '3', '11', '986-belk',
       '842544654-childrensplace', '1718

In [19]:
ret_df[ret_df['Retailer ID'].isnull()] #drop ones where retailers is na

,Market Index,Retail Site,Retailer ID,Crawled?
27,Fast Fashion Index,Loralette,NaN,Crawler
42,Luxury Index,11Honor,NaN,To be Crawled
103,Specialty/DTC Index,Rockets of Awesome,NaN,To be Crawled


In [20]:
ret_df = ret_df.drop([27, 42, 103]) #truncate Off Price Index

In [38]:
#pulling all of it together
#trends = ['neon', 'plaid', 'high%20waisted%20jeans', 'animal print', 'one shoulder', ]
trends = ['neon', 'one%20shoulder', 'paper%20bag%20waist', 'puff%20sleeve', 'high%20waisted%20jeans', 'animal%20print']
retailer_ids = ret_df["Retailer ID"].values
#zara, KHOLS, Moda Operandi, ASOS
dflist_cnt = []
dflist_med=[]
for trend in trends:
    for ret in retailer_ids:
        print(trend, ret)
        api_endpt = 'https://market-api-3.trendalytics.co/v1/products/histogram?filters=sku&cat=womens-clothes'\
            '&fts='+trend+'&fl=r'+ret+'&'\
            'regions=us&pricetype=current&sku_by=week&month_span=20'
        response = requests.get(api_endpt)
        try:
            output = json.loads(response.content.decode('utf-8'))
        except:
            print("Response status code is: ",response.status_code)
            
        count_i, med_i = expand_agg_price(output['skuHistogram'], trend, ret)
        dflist_cnt.append(count_i)
        dflist_med.append(med_i)
    
df_cnt_all = pd.concat(dflist_cnt, axis = 0, sort = True)
df_med_all = pd.concat(dflist_med, axis = 0, sort= True)
    #df_all.to_csv(trend+".csv")
    
            
print('finished')  

neon 864
neon 400331576-isabelmarant
neon 45771179-alc
neon 2412
neon 801
neon 760
neon 152651556-acnestudios
neon 804731108-iro
neon 1909
neon 750312749-thekooples
neon 760
neon 359318405-lagence
neon 105
neon 21
neon 1691
neon 500269677-aritzia
neon 8
neon 25
neon 306
neon 1
neon 23
neon 756639511-seedheritage
neon 574
neon 930080541-massimodutti
neon 398219112-torrid
neon 2001
neon 2294
neon 1690
neon 58533412-zara
neon 108
neon 36
neon 1745
neon 483
neon 408
neon 1689
neon 970193273-rue21
neon 29
neon 1841
neon 491303298-next
neon 2351
neon 2531
neon 27
neon 895
neon 1375
neon 1402
neon 6
neon 7
neon 994
neon 769926722-hermes
neon 10
neon 9
neon 2601
neon 344
neon 625324324-louisvuitton
neon 924989490-prada
neon 562481900-saintlaurent
neon 312443187-valentino
neon 2493
neon 157913149-tomford
neon 900125254-chloe
neon 777583743-dolcegabbana
neon 220138893-dior
neon 1787
neon 494
neon 1340
neon 716752426-walmart
neon 928
neon 413
neon 2
neon 133
neon 150
neon 3
neon 11
neon 986-belk


puff%20sleeve 716752426-walmart
puff%20sleeve 928
puff%20sleeve 413
puff%20sleeve 2
puff%20sleeve 133
puff%20sleeve 150
puff%20sleeve 3
puff%20sleeve 11
puff%20sleeve 986-belk
puff%20sleeve 842544654-childrensplace
puff%20sleeve 171870510-carters
puff%20sleeve 302069529-cvs
puff%20sleeve 61
puff%20sleeve 42
puff%20sleeve 13
puff%20sleeve 190
puff%20sleeve 357261492-credobeauty
puff%20sleeve 887769305-bluemercury
puff%20sleeve 64
puff%20sleeve 101
puff%20sleeve 41
puff%20sleeve 1276
puff%20sleeve 720
puff%20sleeve 5
puff%20sleeve 4
puff%20sleeve 54
puff%20sleeve 1652
puff%20sleeve 1953
puff%20sleeve 2382
puff%20sleeve 2548
puff%20sleeve 207609522-colourpop
puff%20sleeve 311294739-kyliecosmetics
puff%20sleeve 378778715-bonobos
puff%20sleeve 1138
puff%20sleeve 4421110-universalstandard
puff%20sleeve 2506
high%20waisted%20jeans 864
high%20waisted%20jeans 400331576-isabelmarant
high%20waisted%20jeans 45771179-alc
high%20waisted%20jeans 2412
high%20waisted%20jeans 801
high%20waisted%20jeans 

In [39]:
df_cnt_all.tail(5)

,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,ret_id,trend
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,311294739-kyliecosmetics,animal%20print
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,378778715-bonobos,animal%20print
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1138,animal%20print
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4421110-universalstandard,animal%20print
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2506,animal%20print


In [40]:
df_med_all.tail(5)

,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,ret_id,trend
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,311294739-kyliecosmetics,animal%20print
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,378778715-bonobos,animal%20print
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1138,animal%20print
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4421110-universalstandard,animal%20print
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2506,animal%20print


In [41]:
ret_df.columns

Index(['Market Index', 'Retail Site', 'Retailer ID', 'Crawled?'], dtype='object')

In [42]:
df_cnt_all_m = ret_df.merge(df_cnt_all, left_on = 'Retailer ID', right_on ='ret_id')
df_med_all_m = ret_df.merge(df_med_all, left_on = 'Retailer ID', right_on ='ret_id')

In [46]:
df_med_all_m[df_med_all_m["trend"] == "one%20shoulder"]

,Market Index,Retail Site,Retailer ID,Crawled?,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,ret_id,trend
1,Contemporary Index,Alice + Olivia,864,Crawler,NaN,365.0,402.5,357.0,237.0,176.0,...,277.0,277.0,277.0,795.0,795.0,795.0,795.0,795.0,864,one%20shoulder
7,Contemporary Index,Isabel Marant,400331576-isabelmarant,Crawler,NaN,NaN,NaN,NaN,NaN,NaN,...,395.0,395.0,294.0,269.5,245.0,NaN,NaN,NaN,400331576-isabelmarant,one%20shoulder
13,Contemporary Index,A.L.C.,45771179-alc,Crawler,NaN,NaN,NaN,NaN,NaN,NaN,...,295.0,545.0,620.0,620.0,620.0,306.5,248.0,456.5,45771179-alc,one%20shoulder
19,Contemporary Index,Sandro,2412,Crawler,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2412,one%20shoulder
25,Contemporary Index,AllSaints,801,Crawler,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,228.0,124.0,136.0,136.0,114.0,NaN,NaN,801,one%20shoulder
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,Specialty/DTC Index,Kylie Cosmetics,311294739-kyliecosmetics,Crawler,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,311294739-kyliecosmetics,one%20shoulder
595,Specialty/DTC Index,Bonobos,378778715-bonobos,Crawler,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,378778715-bonobos,one%20shoulder
601,Specialty/DTC Index,BaubleBar,1138,Crawler,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1138,one%20shoulder
607,Specialty/DTC Index,Universal Standard,4421110-universalstandard,Crawler,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4421110-universalstandard,one%20shoulder


In [47]:
df_cnt_all_m.to_csv('df_cnt_all_103119.csv')
df_med_all_m.to_csv('df_med_all_103119.csv')

In [1]:
### Nov 29th 2019 data pull ###


In [3]:
search_data = pkl.load(open("./data/search_data_2019-09.pkl", "rb"))

NameError: name 'pkl' is not defined

In [2]:
termDict_2

{'Open Shoulder': ['open shoulder',
  'off the shoulder',
  'off shoulder',
  'cold shoulder'],
 'Satin/Silk': ['satin', 'silk'],
 'Animal Print': ['animal print',
  'leopard',
  'cheetah',
  'tiger',
  'snakeskin',
  'zebra'],
 'Prairie': ['prairie',
  'milkmaid',
  'puff sleeve',
  'puff shoulder',
  'smocked, smocking'],
 'Utility': ['utility', 'cargo', 'boiler suit']}